# **Predict which Tweets are about real disasters**


# Installing Required Packages

In [ ]:
!pip install ktrain

In [ ]:
!pip install contractions

#  Importing Required Packages

In [ ]:
import pandas as pd
import numpy as np
import sys  
import re
import string
import contractions
from sklearn.model_selection import train_test_split
import ktrain
import tensorflow as tf
from ktrain import text

# Data Preparation

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_train

In [ ]:
df_train.dtypes

In [ ]:
df_val = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df_val

In [ ]:
df_train['target'].value_counts(normalize=True)

In [ ]:
sum(df_train.keyword.isna())

In [ ]:
sum(df_train.location.isna())

**Droping keyword and location columns**

In [ ]:
df_train.drop(columns=['keyword', 'location' ,'id'], inplace=True)
df_train

# **Initial Text Pre-Processing**
**We'll remove hashtags(#example), @username and links(starting with http:// or https://) only. As we are going to use BERT, we are not removing emoticons as it will help BERT in prediction. We will again do text pre-processing later using BERT.**

In [ ]:
def pre_process(tweet):
    tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|(#[A-Za-z0-9]+)", " ", tweet).split())  # remove #tags and @usernames
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split()) # remove urls
    return(tweet)

In [ ]:
def pre_process1(tweet):
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split()) # remove urls
    return(tweet)

**Handling constractions**:  Below funnction will replace constactions (e.g. wouldn't to would not).

In [ ]:
def fn_contractions(tweet):
    expanded_words = []
    for word in tweet.split():
        expanded_words.append(contractions.fix(word))
    return(' '.join(expanded_words))

In [ ]:
df_train['text'] = df_train['text'].apply(lambda x:pre_process(x))
df_train

In [ ]:
df_train['text'] = df_train['text'].apply(lambda x:fn_contractions(x))
df_train

In [ ]:
df_val['text'] = df_val['text'].apply(lambda x:pre_process(x))
df_val['text'] = df_val['text'].apply(lambda x:fn_contractions(x))
df_val

# Spliting Data for Test and Train

In [ ]:
train, test = train_test_split(df_train, test_size=0.2)
X_train = train.text.tolist()
X_test = test.text.tolist()
y_train = train.target.tolist()
y_test = test.target.tolist()

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

# Model building using BERT

We are using bert-base-uncased model. You can choose any other model. I am selecting maxlen of tokenization as 512 (it's max for BERT).

In [ ]:
model_arch ='bert-base-uncased'
factors = [0,1] # We have two factors to predict.
MAXLEN = 512
trans = text.Transformer(model_arch, maxlen=MAXLEN, class_names= factors)

In [ ]:
train_data = trans.preprocess_train(X_train,y_train)
test_data = trans.preprocess_test(X_test,y_test)

In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=10)

In [ ]:
#learner.lr_find(show_plot=True, max_epochs=10) #finding optimal learning rate

In [ ]:
learner.fit_onecycle(3e-5, 4)

In [ ]:
learner.validate(val_data=test_data, class_names=factors)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

# Prediction

In [ ]:
df_val['target'] = predictor.predict(df_val.text.tolist())
df_val

In [ ]:
df_val.to_csv('/kaggle/working/test_result_final.csv', index=False)

In [ ]:
df_submission = df_val[['id','target']]

In [ ]:
df_submission.to_csv('/kaggle/working/submission5.csv', index=False)